In [1]:
import pandas as pd
import os

In [2]:
#loading the datas
file_paths = {
    'AAPL': '../data/AAPL_historical_data.csv',
    'GOOGL': '../data/AMZN_historical_data.csv',
    'AMZN': '../data/GOOG_historical_data.csv',
    'MSFT': '../data/META_historical_data.csv',
    'TSLA': '../data/MSFT_historical_data.csv',
    'META': '../data/NVDA_historical_data.csv',
    'NFLX': '../data/TSLA_historical_data.csv'
}

stock_data = {}

for company, path in file_paths.items():
    df = pd.read_csv(path, parse_dates=['Date'])
    df.set_index('Date', inplace=True) 
    stock_data[company] = df


In [3]:
#cleaning - drop any missing datasets

for name, df in stock_data.items():
    stock_data[name] = df.dropna()
    print(f"{name}: {stock_data[name].isnull().sum().sum()} missing values after dropping")


AAPL: 0 missing values after dropping
GOOGL: 0 missing values after dropping
AMZN: 0 missing values after dropping
MSFT: 0 missing values after dropping
TSLA: 0 missing values after dropping
META: 0 missing values after dropping
NFLX: 0 missing values after dropping
